In [54]:
OUTPUT_PATH = '../../data/'

In [55]:
%run '../schema_definition/schema.py'

In [56]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [57]:
from pyspark.shell import spark
from pyspark.sql import functions as F

In [58]:
gold_df = spark.read.json(OUTPUT_PATH + "gold", gold_schema)
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT *
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
fastest_response_df.coalesce(1).write.csv(OUTPUT_PATH + 'response', sep=',', header=True, mode='overwrite')
fastest_response_df.toPandas()

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey,Response
0,10046,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,6,0.00,v1_0,307312,EA983_74004197612-N-11,2017-09-11 08:53:00,RESI,2581,19e635adb1867a76addf356de890d7497199057b6af787...,-1
1,10066,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,5,0.00,v1_0,307312,EA983_78004104538-N-11,2017-09-11 09:25:00,RESI,2581,782974fe34aaaed7094ee399e9d592f17bc1b181da3f06...,-1
2,10035,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,5,21.44,v1_0,307312,EA983_602200346738-N-5,2017-11-29 12:43:00,RESI,2581,b0356a3a9934382308ce87e9cdafa7d8921064d68a9198...,-1
3,10045,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,2.12,v1_0,307312,EA983_146001477714-N-10,2017-09-11 10:35:00,RESI,2581,4dde7d9824a93ea8841b81833ced5c54d5283485f400e1...,-1
4,10040,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,4.15,v1_0,307312,EA983_18005025353-N-1,2017-09-11 10:44:00,RESI,2581,d717bb2d12b4da07f912803c6b2c5714438d43d2e4d32e...,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,10018,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-09-08,PAID,8,-1.75,v1_0,307664,CEN99573813-C-8,2017-09-18 13:13:00,RESI,2582,c1b7a3cfd9da5aadbfed6845a65ead2d9f288de6893b9b...,-368
29649,10047,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-08-29,PAID,4,-38.49,v1_0,307664,CEN98648458-C-8,2017-09-18 13:13:00,RESI,2582,34e79972cc501cbdbe434cbb28bce4b3d0cd6b57729653...,-378
29650,10039,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-07-14,PAID,0,-0.59,v1_0,307662,CEN94977071-C-2,2017-09-18 13:14:00,RESI,2582,260b832faef041ba358c15077b3db8b7ed10855526b1cd...,-424
29651,10037,107,2017-09-11,1.0,B2B,ELE,NUOS,2014-11-26,PAID,7,-10.46,v1_0,307662,CEN51544029-C-2,2017-09-18 13:14:00,RESI,2582,c43daf1377ee14c9b40c34a25eff9b49c03e613f830d33...,-1020
